####Importing required libraries

In [0]:
# for error handling
import sys

# for tokenizeWords
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')

# for tokenizeStemmedWords
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# for tokenizeEmojis
!pip install emoji
import emoji
import re

# for tokenizeUrls
import http.client as httplib
import urllib.parse as urlparse
import requests

import math
import decimal

####Defining function

In [0]:
def tokenizeWords(s, remove_punctuation=True):
  """
  Return array[string] of each word of the text.
  Params:
  s: string to be tokenized
  remove_punctuation: remove punctuation using RegexpTokenizer (default to False)
  """
  if remove_punctuation == True:
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    clean_words = tokenizer.tokenize(s)
  else:
    clean_words = nltk.word_tokenize(s)
  return clean_words

def tokenizeStemmedWords(s):
  """
  Return array[string] of the text with each word has been stemmed to it's root form.
  Params:
  s: text to be processed
  """
  clean_stemmed_words = tokenizeWords(s)
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  for n, word in enumerate(clean_stemmed_words):
    clean_stemmed_words[n] = stemmer.stem(word)
    if clean_stemmed_words[n] == '':
      clean_stemmed_words.pop(n)
  return clean_stemmed_words

def tokenizeEmojis(s):
  """
  Return array[string] of emoji's alias that exist in the text.
  Params:
  s: text to extract emoji's alias from
  """
  emojis_alias = []
  words = re.findall(r'[^\w\s,]', s)
  for word in words:
    if any(char in emoji.UNICODE_EMOJI for char in word):
      emojis_alias.append(word)
  for n, char in enumerate(emojis_alias):
    emojis_alias[n] = emoji.demojize(char)
  return emojis_alias

def tokenizeUrls(s):
  """
  Return array[string] of all expanded urls from all shortened urls that exis in the text.
  Params:
  s: text to extract expanded url from
  """
  urls = re.findall(r'(https?://\S+)', s)
  for n, url in enumerate(urls):
    m = re.search(r',$', url)
    if m != None:
      url = url[:-1]
    try:
      urls[n] = requests.head(url, allow_redirects=True).url
    except Exception as ex:
      print('{0} when handling {1}' .format(sys.exc_info()[0], urls[n]))
  return urls

def tokenizeHashtags(s):
  """
  Return array[string] of all hashtags that exis in the text.
  Params:
  s: text to extract hashtags from
  """
  hashtags = [word for word in s.split() if word.startswith("#")]
  for n, tag in enumerate(hashtags):
    hashtags[n] = re.sub(r"[!?.,]", "", tag)
  return hashtags

def numToWords(num):
  """
  Convert number into their Indonesian words.
  Params:
  num: numnber to convert
  """
  d = {0: 'nol', 1: 'satu', 2: 'dua', 3: 'tiga', 4: 'empat', 5: 'lima',
       6: 'enam', 7: 'tujuh', 8: 'delapan', 9: 'sembilan', 10: 'sepuluh',
       11: 'sebelas', 12: 'dua belas', 13: 'tiga belas', 14: 'empat belas',
       15: 'lima belas', 16: 'enam belas', 17: 'tujuh belas', 18: 'delapan belas',
       19: 'sembilan belas', 20: 'dua puluh', 30: 'tiga puluh', 40: 'empat puluh',
       50: 'lima puluh', 60: 'enam puluh', 70: 'tujuh puluh', 80: 'delapan puluh',
       90: 'sembilan puluh'}
  ribu = 1000
  juta = ribu * 1000
  miliar = juta * 1000
  triliun = miliar * 1000

  if type(num) == str:
    if '.' in num:
      num = float(num)
    else: num = int(num)
    
  if (num < 0):
    return 'minus ' + numToWords(num * -1)

  if type(num) == float:
    float_int, float_dec = FloatToInt(num)
    normalized_numbers = numToWords(float_int) + ' koma'
    for words_num in [numToWords(d) for d in str(float_dec)]:
      normalized_numbers += ' ' + words_num
    return normalized_numbers

  if (num < 20):
    return d[num]

  if (num < 100):
    if num % 10 == 0: return d[num]
    else: return d[num // 10 * 10] + ' ' + d[num % 10]

  if (num < ribu):
    if num // 100 == 1:
      if num % 100 == 0: return 'seratus'
      else: return 'seratus ' + numToWords(num % 100)
    elif num % 100 == 0: return d[num // 100] + ' ratus'
    else: return d[num // 100] + ' ratus ' + numToWords(num % 100)

  if (num < juta):
    if num // ribu == 1:
      if num % ribu == 0: return 'seribu'
      else: return 'seribu ' + numToWords(num % ribu)
    elif num % ribu == 0: return numToWords(num // ribu) + ' ribu'
    else: return numToWords(num // ribu) + ' ribu ' + numToWords(num % ribu)

  if (num < miliar):
    if (num % juta) == 0: return numToWords(num // juta) + ' juta'
    else: return numToWords(num // juta) + ' juta ' + numToWords(num % juta)

  if (num < triliun):
    if (num % miliar) == 0: return numToWords(num // miliar) + ' miliar'
    else: return numToWords(num // miliar) + ' miliar ' + numToWords(num % miliar)

  if (num % triliun == 0): return numToWords(num // triliun) + ' triliun'
  else: return numToWords(num // triliun) + ' triliun ' + numToWords(num % triliun)

  raise AssertionError('num is too large: %s' % str(num))

def normalizeNumbers(s):
  """
  Return string of text with all the number have been converted into words.
  Params:
  s: text to extract number from and convert it to words.
  """
  normalized_numbers_in_text = tokenizeWords(s, False)
  find_number = re.compile("[-+]?\d*?.\d+|\d+")
  for n, word in enumerate(normalized_numbers_in_text):
    num = normalized_numbers_in_text[n]
    num1 = find_number.search(num)
    if num1 != None:
      normalized_numbers_in_text[n] = numToWords(num1.group())
  return normalized_numbers_in_text

def FloatToInt(float_num):
  """
  Split a float number into two integer and decimal part.
  Params:
  float_num: float number
  float_int: integer part of the input float number
  float_dec: decimal part of the input float number.
  """
  if type(float_num) != float:
    float_num = float(float_num)

  float_dec, float_int = math.modf(float_num)

  float_int = int(float_int)
  float_dec = round(float_dec - int(float_dec), 10)
  d = decimal.Decimal(str(float_dec))
  d = d.as_tuple().exponent * -1
  float_dec = int(float_dec * 10**d)
  return float_int, float_dec

####Data for testing

In [0]:
s = """Bagi saya, cerita ini dimulai dari 15 tahun yang lalu.
ketika saya adalah seorang dokter di Universitas Chicago.
Dan saya sedang merawat orang-orang yang sedang sekarat dan keluarganya.
di bagian selatan di Chicago.
Dan saya memperhatikan apa yang terjadi dengan orang-orang dan keluarga mereka.
ketika menerima perawatan untuk penyakit kronis mereka.
Dan di laboratorium saya, saya mempelajari widower effect,
yang merupakan suatu ide lama dalam ilmu sosial,
dari 150 tahun yang lalu.
terkenal dengan sebutan "meninggal karena patah hati."
Jadi, ketika saya meninggal, resiko kematian istri saya dapat meningkat dua kali lipat.
sebagai contoh, di tahun pertama.
Dan saya telah merawat satu orang pasien,
seorang wanita yang terkena penyakit dementia.
Dan dalam kasus ini, tidak seperti pasangan ini,
dia dirawat
oleh anak perempuannya.
Dan anak perempuannya kelelahan karena merawat ibunya.
Dan suami anak perempuan tersebut,
dia juga sakit
dari kelelahan istrinya.
Dan saya sedang menyetir menuju ke rumah,
dan saya menerima telepon dari teman suami tersebut,
memanggil saya karena dia stress
tentang apa yang terjadi dengan temannya.
Jadi saya mendapat telepon dari orang asing ini
menunjukkan suatu kejadian
yang terjadi karena terpengaruh oleh orang lain
pada jarak sosial tertentu.
Dan saya tiba-tiba menyadari dua hal yang sangat sederhana:
Satu, efek widowhood
tidak terbatas pada suami dan istri saja
Dan kedua, tidak terbatas pada pasangan orang-orang terdekat.
Dan saya mulai melihat dunia
dengan cara yang baru,
seperti orang-orang yang terhubung satu sama lain.
Dan saya menyadari bahwa orang-orang ini
terhubung oleh empat orang dari pasangan orang-orang terdekat.
Dan kemudian, sebenarnya, orang-orang ini
terikat dalam hubungan yang lain:
pernikahan dan pasangan hidup
dan pertemanan dan ikatan yang lainnya
Dan sebenarnya, hubungan ini sangatlah luas
dan kita semua terikat di sini
hubungan yang sangat luas satu sama lain."""

emoji_test = "Lorem ipsum 👩🏾‍🎓 dolor sit amet 🤔 🙈, consectetur 😌😊 adipiscing 💕👭👙 elit. 👨‍👩‍👦‍👦 Nunc rutrum. 🙅🏽🙅🏽"

url_test = "Here is an url https://id.wikipedia.org/wiki/Koronavirus, here is a direct url https://www.twitter.com/WatchmenID, and here is another shortened url for you http://bit.ly/ModulBDDA2019."

hashtag_test = "This is a #sample text to test #hashtag extraction for #NLP."

num_int_test = 1212849542
num_str_test = "9827412183"

normal_int = "Normal number: 5131"
minus_int = "Minus number: -207"
normal_float = "Float number:  3.14"
minus_float = "Minus float: -1.5"

####Testing

In [36]:
clean_words = tokenizeWords(s)
clean_stemmed_words = tokenizeStemmedWords(s)
emojis_alias = tokenizeEmojis(emoji_test)
hashtags = tokenizeHashtags(hashtag_test)
word_number = numToWords(num_str_test)
urls = tokenizeUrls(url_test)
normalized_num = normalizeNumbers(normal_int)
normalized_minus_num = normalizeNumbers(minus_int)
normalized_float_num = normalizeNumbers(normal_float)
normalized_minus_float_num = normalizeNumbers(minus_float)

print('tokenizeWords result:')
print('words found: ' + str(len(clean_words)))
print(clean_words)
print('-----------------------------------------------------------------------')
print('tokenizeStemmedWords result:')
print('stemmed words: ' + str(len(clean_stemmed_words)))
print(clean_stemmed_words)
print('-----------------------------------------------------------------------')
print('tokenizeEmojis result:')
print('emojis found : ' + str(len(emojis_alias)))
print(emojis_alias)
print('-----------------------------------------------------------------------')
print('tokenizeHashtags result:')
print('hashtags found: ' + str(len(hashtags)))
print(hashtags)
print('-----------------------------------------------------------------------')
print('numToWords result:')
print('input number: ' + str(num_str_test))
print(word_number)
print('-----------------------------------------------------------------------')
print('tokenizeUrls result:')
print('urls found: ' + str(len(urls)))
print(urls)
print('-----------------------------------------------------------------------')
print('normalized_num result:', normalized_num)
print('normalized_minus_num result:', normalized_minus_num)
print('normalized_float_num result:', normalized_float_num)
print('normalized_minus_float_num result:', normalized_minus_float_num)

tokenizeWords result:
words found: 334
['Bagi', 'saya', ',', 'cerita', 'ini', 'dimulai', 'dari', '15', 'tahun', 'yang', 'lalu', '.', 'ketika', 'saya', 'adalah', 'seorang', 'dokter', 'di', 'Universitas', 'Chicago', '.', 'Dan', 'saya', 'sedang', 'merawat', 'orang', '-orang', 'yang', 'sedang', 'sekarat', 'dan', 'keluarganya', '.', 'di', 'bagian', 'selatan', 'di', 'Chicago', '.', 'Dan', 'saya', 'memperhatikan', 'apa', 'yang', 'terjadi', 'dengan', 'orang', '-orang', 'dan', 'keluarga', 'mereka', '.', 'ketika', 'menerima', 'perawatan', 'untuk', 'penyakit', 'kronis', 'mereka', '.', 'Dan', 'di', 'laboratorium', 'saya', ',', 'saya', 'mempelajari', 'widower', 'effect', ',', 'yang', 'merupakan', 'suatu', 'ide', 'lama', 'dalam', 'ilmu', 'sosial', ',', 'dari', '150', 'tahun', 'yang', 'lalu', '.', 'terkenal', 'dengan', 'sebutan', '"meninggal', 'karena', 'patah', 'hati', '."', 'Jadi', ',', 'ketika', 'saya', 'meninggal', ',', 'resiko', 'kematian', 'istri', 'saya', 'dapat', 'meningkat', 'dua', 'kali', '